In [78]:
import os
import json
import cv2
import xml.etree.ElementTree as Et

from tqdm import tqdm

In [79]:
# data hunt format(4 points[x,y])
c1_path = os.path.expanduser("~/Desktop/joo/data/korad/manager/") # annotation (xml), images 

ai_hub_path = os.path.expanduser("~/Desktop/joo/data/korad/data/origin")
ai_hub_type = ["images/","labeling/"] # 01 ~ 06 (json) / cf. 01 is empty

### rename .jpeg.json -> .json

In [80]:
for number in os.listdir(ai_hub_path+ai_hub_type[1]):
    path = ai_hub_path+ai_hub_type[1]+number+'/'
    files = next(os.walk(path))
    
    print(f"folder {number} start!")
    for index, file in tqdm(enumerate(files[2])):
        
        if file[-9:] != "jpeg.json": # ai_hub 데이터를 업체에서 넘긴게 .jpeg.json으로 보내줌.
            print(f"[Warning!] folder {number} have already done!\n======================================")
            break

        os.rename(path+file,path+file[:-9]+"json")

folder 03 start!


0it [00:00, ?it/s]


[Warning!] folder 03 have already done!
folder 05 start!


0it [00:00, ?it/s]


[Warning!] folder 05 have already done!
folder 02 start!


...

folder 06 start!


0it [00:00, ?it/s]


[Warning!] folder 06 have already done!
folder 04 start!


0it [00:00, ?it/s]

[Warning!] folder 04 have already done!


### xml file -> json !!

In [92]:
path,_,c1_files = next(os.walk(os.path.join(c1_path+"annotation/"))) # annotation, json
img_path,_,c1_img_files = next(os.walk(os.path.join(c1_path+"images/"))) # annotation, json
c1_files.sort()
c1_img_files.sort()

print(f"c1_ano : {len(c1_files)}, c1_img : {len(c1_img_files)}")
empty_list = {}

for index, file in tqdm(enumerate(c1_img_files)):
    save_file_name = file
    file = "C1"+file[file.index('_'):]
    
    if os.path.exists(path+"C1"+file[file.index('_'):-4]+".xml") == False:
        empty_list[file[:-4]] = ["01",file, file[:-4]+".xml"]
        continue
    
    # if os.path.exists(path+file[:-4]+".json") == False:
    #     empty_list[file[:-4]] = ["01",file, file[:-4]+".json"]
    #     continue

    # continue # 제거하면 json 파일 만듬
    file = file[:-4]+".xml"
    
    root = Et.parse(open(os.path.join(path, file), "r")).getroot()

    filename = root.find("filename").text
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    objects = root.findall("object")
    
    json_data = {}
    
    json_data['images'] = {'id':filename[:-4],"width":width,"height":height,"file_name":filename}
    json_data['labels'] = []

    for _object in objects:
        name = _object.find("name").text
        bndbox = _object.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        
        coordinate = [{"xmin":xmin,"ymin":ymin,"xmax":xmax,"ymax":ymax}]
        json_data['labels'].append({"label": name, "boundingPoly":{"type":"RECT", "vertices":coordinate}})
    
    with open(os.path.expanduser(c1_path+"json/"+save_file_name[:-4]+".json"), 'w', encoding='utf-8') as file:
        json.dump(json_data, file)
print(len(empty_list))

c1_ano : 2993, c1_img : 3900


3900it [00:00, 8605.10it/s]

907


In [77]:
print(f"짝 안맞는 파일 개수 : {len(empty_list)}")
with open(os.path.expanduser("./korad_boxes/error_list2.json"), 'w', encoding='utf-8') as f:
    json.dump(empty_list,f)

짝 안맞는 파일 개수 : 907


In [ ]:
ai_img = cv2.imread(os.path.expanduser("~/Desktop/joo/data/korad/data/origin/images/01/101_6a1d3941-0615-11ec-919c-a85e4515a1df.tiff"))

manager_img = cv2.imread(os.path.expanduser("~/Desktop/joo/data/korad/manager/images/C1_00232.jpg"))

In [ ]:
print(manager_img[0][10])
print()
print(ai_img[0][10])
print(manager_img[0][10] == ai_img[0][10])